<a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

# 物体検出

### デプロイの活用

スライディングウィンドウを用いて物体を検出しようとすることは、1 枚の画像の 256 x 256 部分を「dog」または「cat」として分類する画像分類器を、一度にひとつの 256 x 256 領域を処理するアプリケーションへデプロイすることを含みます。
これは完璧なソリューションではありませんが、以下を行います。

1) ディープラーニングと従来のプログラミングを融合して、以前では不可能だったアプリケーションを生成するための効果的な方法の実例を示す。  
2) ニューラルネットワークアーキテクチャーについて学習するための開始点に導く。  

このコースの最初のセクションで、ニューラルネットワークの *学習* と *デプロイ* を正しく行う方法を学びました。
従来のプログラミングが画像の分類には不適当であることが判明しましたが、ディープラーニングであれば単に実行できるだけでなく、容易に行えるのです。

これまでに、既存のニューラルネットワークとオープンソースデータセット（dogs vs. cats）を取り上げ、あなたの初期のデータセットに似た、ならびに似ていない新しい動物を正しく分類することができるモデルを生成しました。
とてもパワフルですね！

**この** タスクでは、ディープラーニングにより *画像の分類を超えた* 問題の解き方について、以下のように学びます。

- ディープラーニングと従来のコンピュータービジョンを融合する
- ニューラルネットワークの内部（実際の数学）を改変する
- ジョブにとって適切なネットワークを選定する

こうすることで、ディープラーニングに関する第二の課題を解決します。

**画像の中でオブジェクトを *検出* し、 *場所を特定* することができるのか？**

前のセクションで行ったときと同じ方法で、まずは私たちのモデルのインスタンスを作成してみましょう。
モデルのアーキテクチャ、学習済みの重み、および平均画像など、前処理情報を利用するため、モデルとデータセットのジョブディレクトリを確認します。

In [ ]:
import time
import numpy as np #Data is often stored as "Numpy Arrays"
import matplotlib.pyplot as plt #matplotlib.pyplot allows us to visualize results
import caffe #caffe is our deep learning framework, we'll learn a lot more about this later in this task.
%matplotlib inline

MODEL_JOB_DIR = '/dli/data/digits/20180301-185638-e918'  ## Remember to set this to be the job directory for your model
DATASET_JOB_DIR = '/dli/data/digits/20180222-165843-ada0'  ## Remember to set this to be the job directory for your dataset

MODEL_FILE = MODEL_JOB_DIR + '/deploy.prototxt'                 # This file contains the description of the network architecture
PRETRAINED = MODEL_JOB_DIR + '/snapshot_iter_735.caffemodel'    # This file contains the *weights* that were "learned" during training
MEAN_IMAGE = DATASET_JOB_DIR + '/mean.jpg'                      # This file contains the mean image of the entire dataset. Used to preprocess the data.

# Tell Caffe to use the GPU so it can take advantage of parallel processing. 
# If you have a few hours, you're welcome to change gpu to cpu and see how much time it takes to deploy models in series. 
caffe.set_mode_gpu()
# Initialize the Caffe model using the model trained in DIGITS
net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

# load the mean image from the file
mean_image = caffe.io.load_image(MEAN_IMAGE)
print("Ready to predict.")

次に、玄関のセキュリティカメラから画像を直接取得します。
この画像の大きさは、256 x 256 を超えていることに注意してください。
犬は、この画像の *どこ* にいるのでしょうか。

In [ ]:
# Choose a random image to test against
#RANDOM_IMAGE = str(np.random.randint(10))
IMAGE_FILE = '/dli/data/LouieReady.png'
input_image= caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

### *学習* された関数の利用：順伝播

上記の画像サイズの問題は、気をつけるべき事柄です。
関数を見てみましょう。

<code>prediction = net.predict([grid_square])</code>

あらゆる[関数](https://www.khanacademy.org/computing/computer-programming/programming#functions)と同様に、<code>net.predict</code> は入力、<code>grid_square</code> を渡し、出力、<code>prediction</code> を戻します。
他の関数とは異なり、この関数はステップの一覧に従わず、層ごとに行列計算を実行して画像を確率ベクトルに変換しています。

以下のセルを実行し、無作為で選んだ画像の左上 256 x 256 <code>grid_square</code> からの予測結果を確認します。

In [ ]:
X = 0
Y = 0

grid_square = input_image[X*256:(X+1)*256,Y*256:(Y+1)*256]
# subtract the mean image (because we subtracted it while we trained the network - more on this in next lab)
grid_square -= mean_image
# make prediction
prediction = net.predict([grid_square])
print prediction

これは、私たちが使用しているネットワークの最後の層の出力です。
層の種類は「softmax」で、各セルの値が高ければ高いほど、そのクラスに画像が所属する確率は高くなります。
たとえば、ベクトルの第二のセルが第一のセルよりも高い場合（そしてネットワークが学習済みである場合）、画像のそのセクションに *dog* が含まれている可能性が高くなります。
今回の場合、第一のセルは *かなり* 高いため、ネットワークが左上の 256 x 256 のグリッドで犬を検出しないのは至極明らかです。

オプション: [Explore the math of softmax.](https://en.wikipedia.org/wiki/Softmax_function)

次に、画像を繰り返し処理し、それぞれの grid_square を分類してヒートマップを生成するため、私たちの関数の周囲に何らかのコードを実装しましょう。
このセクションで主に学ぶことは、この関数の周囲に 「何でも」 構築することができるという事実です。
限界があるとすれば、それはあなたの創造力の大きさによって決定付けられます。

あなたが持つプログラミング（具体的には Python）の経験に応じて、以下のセルから得る情報は大きく異なる可能性があります。
基本的に、このブロックは私たちの入力画像を複数の正方形（256 x 256）に切り、それぞれを犬の分類器で分類し、その判断に従って、犬がいない場所を青色、そして犬がいる場所を赤色とする新しい画像を 1 つ生成します。

In [ ]:
# Load the input image into a numpy array and display it
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

# Calculate how many 256x256 grid squares are in the image
rows = input_image.shape[0]/256
cols = input_image.shape[1]/256

# Initialize an empty array for the detections
detections = np.zeros((rows,cols))

# Iterate over each grid square using the model to make a class prediction
start = time.time()
for i in range(0,rows):
    for j in range(0,cols):
        grid_square = input_image[i*256:(i+1)*256,j*256:(j+1)*256]
        # subtract the mean image
        grid_square -= mean_image
        # make prediction
        prediction = net.predict([grid_square]) 
        detections[i,j] = prediction[0].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections, interpolation=None)

# Display total time to perform inference
print 'Total inference time: ' + str(end-start) + ' seconds'

繰り返しになりますが、これはランダムに選ばれた大きなテスト画像に対する出力です。
モデルに入力される重複のないそれぞれの 256 x 256 のグリッドに対して、クラスを予測した結果を含む配列と関連しています。

どうすればより良く行えたのかを考える前に、以下の問いに答えて今私たちが行ったことについてしばらく考察してみましょう。

**物体を検出するため、どのように画像分類器を使用したか、自分の言葉で説明してください。**

このセクションは、あらゆるディープラーニングのソリューションが、入力から出力までのマッピングに過ぎないことを示すことが目的です。
これに気づくことで、私たちはより複雑なソリューションを構築する方法への理解に、一歩近づくことになります。
たとえば、Alexa、Google Assistant、Siri などの音声アシスタントは、生の音声データをテキストに、テキストを理解に、理解をお気に入りの歌の再生のような求められるタスクに、変換しなければなりません。

<a id='question1'></a>

複雑さは、コンピューターサイエンスによって構築可能で、これまでにも構築されてきた、到達可能なものです。
符号化や問題解決の課題として、このソリューションを構築するときに私たちが考えようとしなかった要因はなんでしょうか？
また、私たちはどのようにしてそれらについて説明することができますか？

想定解答: [ここをクリックしてください](#answer1)

### 1.4 オプションのエクササイズに挑戦

<a id='question-optional-exercise'></a>

1. グリッドの大きさを 256 x 256 に維持して、コードを変換してグリッド間の重複を増やし、より微細な分類マップを取得します。  
2. predictionのため、複数のグリッドをまとめて一つのバッチとなるよう、コードを編集します。  

解答例: [ここをクリック](#answer-optional-exercise)

このように、このスライディングウィンドウアプローチの利点は、（より広く入手することができる）パッチベースの学習データのみを使用して検出器を学習することができ、現在のスキルセットを使用して問題を解決するための作業を開始できることです。

私たちはコードの微調整を続け、（不正確で低速な）総当たり方式で問題の解決を続けられるか、確かめることができます。
もしくは、私たちはディープラーニングについてより深く学ぶこともできます。
後者を選ぶことにしましょう。

## アプローチ 2 ー 既存のニューラルネットワークから再構築する

ディープニューラルネットワークは、人の脳をモデルに作られていることを思い出してください。
このセクションでは、ネットワークに *変更* を加えて、その挙動とパフォーマンスを変更する方法について解説します。
ニューラルネットワークは、根本的には数学的手法によって成り立っており、その手法に変更を加えることで脳に変更が加わります。
このセクションでは、各種のネットワークが各種の問題に対して *なぜ* 理想的であるのか、詳しくは扱いません。
このことについては、時間の経過とともに直感的に分かるようになるでしょう。
代わりに、このセクションでは実験時に留意するべき制約について取り上げます。
一般的ないくつかの種類の層について紹介しますが、それらはこのコースで学ぶべき重要な点ではありません。

ではこれから、私たちの現在のネットワーク、AlexNet について詳しく探ってゆきましょう。

### 2.1 現在のネットワーク

ネットワークの構造は、いずれかの *フレームワーク*（今回は Caffe）において記述されます。
[数多くのフレームワーク](https://developer.nvidia.com/deep-learning-frameworks) が存在しており、こうしたフレームワークの恩恵により、実務家は抽象的なレベルでネットワークを記述できるのに対して、物理的にはテンソル計算を行うために GPU をプログラムしなければなりません。

DIGITS に戻りましょう。

## [ここから DIGITS を起動](/digits)

AlexNet の Caffe に関する説明を DIGITS で確認するには、以下の手順を行ってください。

1) 「Dogs vs. Cats」というモデルを選択します。  
2) すべての設定を新しいネットワークにコピーするため、 "Clone Job"（右上の青色で示された部分）を選択します。ネットワークのクローンは、実験を繰り返し行うための重要な方法です。  
3) AlexNet を選択したモデルの設定の最下部で、 "Customize" を選択します。  

![](images/customize.png)

すると *prototext* が表示されます。
これは、AlexNet ネットワークの Caffe の記述です。
自由に確認して、どのような構造になっているのか理解してください。
他のフレームワークに類似のものが存在しますが、それらの構文は異なっています。

次に、ネットワークの最上部で "Visualize" を選択して、文字通り AlexNet のネットワークを可視化します。

![](images/visualize.png)

ここに存在するものは、あなたにとって大きな意味を持たないかも知れませんが、第一に指摘しておくべきことは、prototext に書かれていることが可視化によって表示される内容を直接表現しているということです。
確認してみてください。
最初の層の *名前* を *train-data* から *your name* に変更し、再び "*Visualize*" を選択してください。

いかがでしょうか。

さて、最初の層の名前を *train-data* に戻し、重要な要素である計算方法を変更しましょう。

新しい機能を持てるよう、AlexNet のネットワークのいくつかの層を置き換え、そして、ネットワークがひきつづき学習できるか確認します。

### 2.2 脳手術 (のようなこと)

1 つの層を置き換えることから始めて、すべてが正しく接続されていることを確かめてください。

prototext の中で、「fc6」という名前の層を見つけてください。
この層が *何を* 行うのかについて、多くの情報を確認できます。
私たちには、いずれ分かります。
今のところは、それを置き換えてみましょう。
"layer" と、次の "layer" の前にある閉じる側の[角括弧](#bracket "}")の間にあるすべての prototext を、以下の内容で置き換えます。

```
layer {
  name: "conv6"
  type: "Convolution"
  bottom: "pool5"
  top: "conv6"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 4096
    pad: 0
    kernel_size: 6
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
```

私たちの新しいネットワークと元の AlexNet と比較するため、ネットワークを *可視化* してみましょう。
参考までに、オリジナルのネットワークの始点は以下のようになっていました。

![](images/AlexNet%20beginning.PNG)

そして、終点は以下のとおり。

![](images/AlexNetEnd.PNG)

これは、私たちが何かを壊したことを示しているでしょうか？

私たちが壊したものを修正するために、何ができるでしょうか？

### 2.3 ジョブの終了

#### ルール 1：データは流れなければならない

私たちは、fc6 の層を除去しました。
しかし、私たちがネットワークを可視化すると、明らかに除去したはずの層が残っています。
これは以下のように *fc6 に言及する他の層が存在している* からです。

- fc7
- drop6
- relu6

それらの層が fc6 のどこに接続するように *指示して* いるのか確認するため Caffe の prototext を眺め、代わりに新しい層である <code>conv6</code> に接続します。

**再び可視化します。**
ネットワークは再び、データが元の AlexNet と同じ入力から出力へ流れることを示すはずです。

データが入力から出力へ流れる際、なんらかの操作（計算）を実行する「隠れ層」が数多く存在します。
各層の出力は、次の入力として定義される必要があります。
これらは大変複合的な機能を生み出します。
こうした機能によって、ディープラーニングは入力と出力の間にある、非常に複雑な実世界の関係性をモデル化することが可能となります。

その一方で、接続関係を維持する限り、任意の計算を使用することができます。
これによりデータは入力から出力へと流れることができます。
他の場合を見てみましょう: 

#### ルール 2：数学的手法の重要性

では、私たちが除去した層と追加した層を比較してみましょう。

私たちは、従来型の行列の乗算である「全結合 (fully connected)」層を除去しました。
行列の乗算については知るべきことがたくさんある一方、私たちが対処するであろうものは、行列の乗算が特定の大きさの行列についてのみ作用するという特性です。
私たちの課題に対してこのことが持つ意味合いは、固定されたサイズの画像に入力が制限されているという事実です（私たちの場合、256 x 256）。

私たちは「畳み込み (convolutional)」層を追加しましたが、これは入力行列上を移動する「フィルター」機能です。
畳み込みについても知るべきことがたくさんある一方、私たちが利用するのは、それらが特定の大きさの行列についてだけ作用するわけではない、という特性です。
私たちの課題に対してこのことが持つ意味合いは、すべての全結合層を畳み込み層で置き換えると、あらゆるサイズの画像を受け入れることができるという事実です。

**AlexNet を「全畳み込みネットワーク (Fully Convolutional Network)」に変換することで、私たちはさまざまなサイズの画像をグリッドに分割することなく、ネットワークに送り込むことができます。**

AlexNet を全畳み込みネットワーク（紛らわしいので「FCN」と表記します）に変換してみましょう。
`fc7` から `fc8` までを、以下の Caffe テキストを使用して同等の畳み込み層と置き換えてください。

**重要：この置き換えは、「fc8」の定義の末尾において止めるべきです。**

```
layer {
  name: "conv7"
  type: "Convolution"
  bottom: "conv6"
  top: "conv7"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 4096
    kernel_size: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu7"
  type: "ReLU"
  bottom: "conv7"
  top: "conv7"
}
layer {
  name: "drop7"
  type: "Dropout"
  bottom: "conv7"
  top: "conv7"
  dropout_param {
    dropout_ratio: 0.5
  }
}
layer {
  name: "conv8"
  type: "Convolution"
  bottom: "conv7"
  top: "conv8"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 2
    kernel_size: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
```

引き続きすべてが適切に接続されていることを保証するため、私たちがネットワークに加えなければならない変更は他にありますか？
確信が持てないときは、ネットワークを再び *可視化* して評価の助けとしてください。
ヒントが必要なときは、[ここ](#rewiring "`loss` と `accuracy` と `softmax` の各層について、 `bottom` ブロブを `fc8` から `conv8` に変更することをお忘れなく")にカーソルを合わせてください。

すべてが適切に接続されていると確信したら、新しいモデルの学習を始めましょう！

![](images/createmodel.PNG)

あなたのモデルに名前を付け、*Create* を選択してください。
学習の間、以下の動画を見て、あなたがネットワークに加えた畳み込みについて、より深く学んでください。

[![What are convolutions?](images/Convolutions.png)](https://youtu.be/BBYnIoGrf8Y?t=13"What are convolutions.")

エラーなく動作しましたか？すばらしい！
もしエラーがあるときは、その内容を確認してください。
どこで何かミスを犯したのかが分かります。
ありがちなミスは[こちら(カーソルを合わせると詳細が見えます)](#rewiring "`loss` と `accuracy` と `softmax` の各層について、 `bottom` ブロブを `fc8` から `conv8` に変更することをお忘れなく")です。
答えが欲しいときは、[ここ](#gu)をクリックしてください。

<a id='fcn'></a>

**モデルの学習が完了したら、最初のセクションで行ったように、以下の ##FIXME## をあなたの新しい「Model Job Directory」で置き換えてください。**

In [ ]:
JOB_DIR = '##FIXME##'  ## Remember to set this to be the job directory for your model

自由にコードを確かめて、このモデルを使用することと、そのままの AlexNet を使用することの違いを知ってください。
ただし、私たちが学ぶべき大切なことは、グリッドに代えて画像 *全体* を私たちの学習済みモデルに与えることです。

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import caffe
import copy
from scipy.misc import imresize
import time

MODEL_FILE = JOB_DIR + '/deploy.prototxt'                 # Do not change
PRETRAINED = JOB_DIR + '/snapshot_iter_735.caffemodel'    # Do not change                 

# Tell Caffe to use the GPU
caffe.set_mode_gpu()

# Load the input image into a numpy array and display it
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

# Initialize the Caffe model using the model trained in DIGITS
# This time the model input size is reshaped based on the randomly selected input image
net = caffe.Net(MODEL_FILE,PRETRAINED,caffe.TEST)
net.blobs['data'].reshape(1, 3, input_image.shape[0], input_image.shape[1])
net.reshape()
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

# This is just a colormap for displaying the results
my_cmap = copy.copy(plt.cm.get_cmap('jet')) # get a copy of the jet color map
my_cmap.set_bad(alpha=0) # set how the colormap handles 'bad' values

# Feed the whole input image into the model for classification
start = time.time()
out = net.forward(data=np.asarray([transformer.preprocess('data', input_image)]))
end = time.time()

# Create an overlay visualization of the classification result
im = transformer.deprocess('data', net.blobs['data'].data[0])
classifications = out['softmax'][0]
classifications = imresize(classifications.argmax(axis=0),input_image.shape,interp='bilinear').astype('float')
classifications[classifications==0] = np.nan
plt.imshow(im)
plt.imshow(classifications,alpha=.5,cmap=my_cmap)
plt.show()

# Display total time to perform inference
print 'Total inference time: ' + str(end-start) + ' seconds'

上記のコードを複数回実行すると、しばしば、全畳み込みネットワークが、スライディングウィンドウのアプローチよりも高い精度で犬の場所を特定することができるのが分かります。
また、FCN のための合計推論時間はおよそ 0.8 秒であった一方、スライディングウィンドウのアプローチでは、その時間は 2.6 秒でした。
私たちは、何分の一かの時間でより質の高い結果を得ることができます。
たとえば航空機上など、リアルタイムの用途で検出器をデプロイしたい場合、これは大きな利点となります。
なぜなら私たちは、一回の効率的なパスで検出と分類を行える単一のモデルを得られるからです。

しかし、犬の部分を見落としたり、犬ではないものを誤って検出してしまったりすることがあります。
背景にある様々な物体や、犬により引き起こされる誤検出は、適切なデータオーグメンテーションをはじめとする、ニューラルネットワークとは別に行われるデータに関するさまざまな処理を用いて緩和することができます。

ネットワークの中で、このような問題をさらに解決することができるでしょうか？

シンプルに言えば、Yes です。

詳しく説明します。
犬を検出し、その場所を特定するよう *設計* された、エンドツーエンドの物体検出ネットワークを使用することで解決できます。
私たちは [COCO](http://cocodataset.org/#home) データセットを用いて学習された DetectNet を利用します。
作業を進める前に、次のセルを実行してメモリを解放します。

In [ ]:
try:
    del transformer
    del net
    del detections
except Exception as e:
    print e

## アプローチ 3：DetectNet

このラボは、まず AlexNet から始まりました。
これは *画像分類* という特定の問題に対する、エレガントな解です。
私たちは AlexNet を中心とした Python のプログラムを構築し、小規模な変更を加え、ディープラーニング以前は不可能であったタスクを達成しました。そのタスクは *物体検出* です。

しかし、*物体検出* のためのエレガントな解は存在するのでしょうか？
私たちは、手元にある入力、すなわちさまざまなサイズの写真から、必要な出力、すなわち位置の特定と検出に関する情報へ、直接的にマッピングするモデルを構築することができるのでしょうか？

ある事実が、私たちがディープラーニングの定義を広めるのを後押ししてくれ、ディープラーニングによって解決できる *他の* 問題への洞察をもたらしてくれます。

このコースは、ディープラーニングを可能にする 3 つの構成要素についての解説から始まりました。

1) ディープニューラルネットワーク  
2) GPU  
3) ビッグデータ  

画像分類を越えて、これらの構成要素は変わりません。
しかし、ネットワークの種類、データ、GPU の使用状況は変化します。
DIGITS 上のワークフローの構成に即して、私たちはデータから開始します。

### データの違い

エンドツーエンド教師ありディープラーニングのワークフローを構築するには、ラベル付けされたデータが必要となります。
ここまで私たちは、「ラベル付け」を各画像が帰属するカテゴリーの指標として定義していました。
しかし、ラベルありデータの実際のタスクは、**入出力のマッピングを作成することです。**

この場合、入力を任意のサイズの画像として、出力が画像中にある物体の位置を示すようにしたいと思っています。
まず、入力画像は以下のとおりです。

In [ ]:
input_image = caffe.io.load_image('/dli/data/train/images/488156.jpg') #!ls this directory to see what other images and labels you could test
plt.imshow(input_image)
plt.show()

次に、それに対応するラベルです。

In [ ]:
!cat '/dli/data/train/labels/488156.txt' #"cat" has nothing to do with the animals, this displays the text

注：

1) 入力とそれに対応する出力は、ファイル番号に基づいて相互に関連付けられています。  
2) あなたが目にしているベクトルは、入力画像内の犬の左上と右下の角の座標 (x,y) で構成されています。  
3) サーフボードについて十分なデータを持っているなら、犬の検出器の代わりにサーフボードの検出器を学習することができます。私たちは、犬を探すことのみを目的にデータセットを設定しています。  

このデータセットを DIGITS に読み込みましょう。

[DIGITS](/digits) のホーム画面から、"Datasets" を選択し、新しい **"Object Detection" データセット** を加えます。
パラメーターは、分類のときとは異なって見えるのでご注意ください。

"New Object Detection Dataset" パネルが開いたら、以下の画像から前処理オプションを選んでください（**前後の余白にご注意ください**）：

Training image folder: /dli/data/train/images  
Training label folder: /dli/data/train/labels  
Validation image folder: /dli/data/val/images  
Validation label folder: /dli/data/val/labels/dog  
Pad image (Width x Height): 640 x 640  
Custom classes: dontcare, dog  
Group Name: MS-COCO  
Dataset Name: coco-dog  

![OD data ingest](images/OD_ingest.png)

注：データセットが読み込みを完了するのを待つことなく、モデルの学習ジョブの設定を開始しても構いません。

### ネットワークの違い

さまざまな種類のことがらを *学習* するために設計された、さまざまな種類の人工「頭脳」が存在します。
あなたの課題と類似している問題を解決するためにどのようなアーキテクチャが用いられているか学ぶことは、ディープラーニングを成功させるための核となるスキルの一つです。
アプローチ 2 では、いくつかの制約を変更するだけで、ネットワークの挙動を変えてしまう可能性があることを目にしました。
アプローチ 3 では、設計者でなくてもこれらのネットワークとモデルを使用することができることに注意しつつ、最新の研究や専門知識、および繰り返し計算することがもたらすメリットを示します。
私たちが利用するネットワークの設計者が直面した課題とは、**データから入出力のペアをマッピングするための最も効率的で正確な方法とは何か** ということでした。

ネットワークを読み込み、可視化することで、これらのネットワークがいかに多様性に満ちたものであるか把握してみましょう。

画面左上で "DIGITS" を選択して、DIGITS のホーム画面に戻ってください。
"Image" -> "Object Detection" の順に選択して、新しい Object Detection **モデル** を作成してください。

あなたがさきほど読み込んだデータセット、"coco-dog" を選択してください。

以下の理由により、エポック数を 1 に変更し、学習率を 1e-100 に変更してください。
通常、ネットワークを選択するときは、"Custom Network"を選択します。
空のテキストエリアが開くはずです。

私たちは、"DetectNet deploy.prototxt" と検索することによって、このネットワークの定義を見つけることができます。
あなたがやるべきことは、このリンク：[deploy.prototxt](../../../../edit/tasks/task5/task/deploy.prototxt) からのテキストを、表示されたフィールドにコピーして貼り付けることです。
このとき、ネットワークのアーキテクチャーを共有することがこれほどまでに容易であることに留意してください。

次に、"visualize" を選択してネットワークを可視化してください。
このネットワークの大きさと複雑さが、 AlexNet のそれとは異なることに注意してください。
DetectNet は実際には、私たちが上で構築したような全畳み込みネットワーク（FCN）ですが、このデータ表現をその出力として正確に生成するように構成されています。
DetectNet の多くの層は、よく知られた GoogLeNet ネットワークと同一です。

まだ学習を始めないでください！

### 計算の違い

ネットワークの深さとタスクの複雑さが高まると、これまで作業してきたネットワークと比較してより *多くの学習* が必要となります。
このワークフローの威力を示すために使用するモデルは、 NVIDIA Tesla K80 上で学習するのにおよそ 16 時間かかりました（Volta ではこれよりも短く、CPU では人生を捧げることになります）。

このため、最初から学習を行う代わりに、事前学習済みのモデルから始め、とても遅い（小さい）学習率（前のステップですでに設定しているもの）で 1 エポックだけ学習します。

学習済みの重みを読み込むには、重みを <pre>/dli/data/digits/snapshot_iter_38600.caffemodel</pre> のファイルと共に <code>Pretrained Model(s)</code> のフィールドに読み込みます。

[](images/loadsnapshot.PNG)

自分のモデルに名前を付け、それを「学習」して、画像分類と物体検出の違いを確認します。

モデルの学習を行うと、画像分類モデルのために報告されたもとのとは異なるパフォーマンスの測定値が確認できます。
平均精度（mAP）とは、ネットワークがどれほど良好に犬を検出することができるか、ならびにそのバウンディングボックス（位置特定）の推定が検証データセットに対してどれほど正確かに関する複合的な測定値です。

100 エポック後に、このモデルが低い学習/検証損失の値に収束したことだけでなく、高い mAP スコアを確認できます。
この学習済みモデルをテスト画像と比較してテストし、犬を見つけることができるか確認してみましょう。

**可視化方法("Select Visualization Method")を "Bounding Boxes" に設定** し、画像パス `/dli/data/BeagleImages/LouieTest1.jpg` を "Image Path" にペーストし、"Test One" をクリックしてください。
DetectNet が Louie を検出し、バウンディングボックスを描く様子が確認できるはずです。

試すべき他の画像の一覧にアクセスするには、以下のセルを実行してください。
Jupyter ノートブックの "!" は、こうした各セルをコマンドラインと同様に振る舞わせることにご注意ください。

In [ ]:
!ls /dli/data/BeagleImages

`dli/data/BeagleImages` フォルダーからの他の画像を自由にテストしてください。
あなたは、DetectNet が、厳密に描かれたバウンディングボックスで多くの犬を正確に検出することができ、誤報率がとても低いことがわかるでしょう。
このモデルは、複数のクラスの物体を検出することができるという点も、注目に値します！

ここで学ぶべきことがいくつかあります。

1. 入手可能ななんらかのジョブ向けの適切なネットワークとデータは、独自のソリューションをハックするより相当に優れています。
2. 適切なネットワーク（および、時には事前学習済みモデル）は、DIGITS または利用しているフレームワークの [Model Zoo](https://github.com/BVLC/caffe/wiki/Model-Zoo) から使用できる可能性があります。

このため、利用可能な1つの問題解決フレームワークは、

- 誰かがすでに、あなたの課題を解決しているかどうか判断した場合、そのモデルを使用します。
- まだ誰もあなたの課題そのものを解決していない場合で、誰かがすでに、あなたの課題に類似した課題を解決したと判断した場合、彼らのネットワークと手持ちのデータを使用します。
- 誰もあなたの課題自体や類似の課題を解決しておらず、あなたの課題を解決するための誰かの解決策の欠点を特定し、新しいネットワークの設計に取り組むことができるか判断します。
- それを行えない場合、既存の解決策、その他の課題解決の技術（Pythonなど）を使用して手持ちの課題を解決します。
- いずれにしても、実験を続け、ラボを利用してスキルを向上させてください！

次に進む前に、これまでに学んだことを振り返ってみましょう。

すでに以下のことを習得済みです。

- DIGITS を使用してさまざまな種類のニューラルネットワークを学習する方法。
- どのような種類の変更が、パフォーマンスを向上させるために、ネットワークを学習する方法に加えることができるか、ということ。
- 新しい課題を解決するために、ディープラーニングと従来的なプログラムを組み合わせる方法。
- 機能と制約を変更するために、ネットワーク内部の層を変更する方法。
- あなたの課題に類似した課題に取り組んだ人たちを確認し、彼らのソリューションのどの部分をあなたが利用できるのかを調べること。

次に、あなたが取り組み始めている課題を解決するために、ディープニューラルネットワークを本番環境に *デプロイ* する方法について、さらに深く学びましょう。
あなたが必要とするであろう情報やデプロイ時のパフォーマンスに関する基準、およびエンドユーザーが管理しているディープラーニングの力と速さを維持する（もしくは、高める）ツールについて、さらに理解を深めましょう。

<a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

## 問題の解答：

<a id='answer1'></a>
### 解答1

私たちが重複しないグリッドのスライディングウィンドウを使用したのは、私たちのグリッドのいくつかには犬が部分的にしか含まれておらず、分類の失敗につながる可能性があるからです。
しかし、グリッドにおいて重複する部分を増やしてゆくと、このスライディングウィンドウのアプローチのための計算時間が急速に増えることになります。
私たちは、入力を *バッチ化* することで、計算におけるこの上昇を軽減することができます。
これは、GPU が持つ並行処理の本質的な力を利用した戦略です。

[ここをクリック](#question1)して、ラボに戻ってください。

<a id='answer-optional-exercise'></a>

### オプション課題への解答

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import caffe
import time

MODEL_JOB_NUM = '##FIXME##'  ## Remember to set this to be the job number for your model
DATASET_JOB_NUM = '##FIXME##'  ## Remember to set this to be the job number for your dataset

MODEL_FILE = MODEL_JOB_NUM + '/deploy.prototxt'                 # Do not change
PRETRAINED = MODEL_JOB_NUM + '/snapshot_iter_735.caffemodel'    # Do not change
MEAN_IMAGE = DATASET_JOB_NUM + '/mean.jpg'                      # Do not change

# load the mean image
mean_image = caffe.io.load_image(MEAN_IMAGE)

# Choose a random image to test against
#RANDOM_IMAGE = str(np.random.randint(10))
IMAGE_FILE = '/dli/data/LouieReady.png' 

# Tell Caffe to use the GPU
caffe.set_mode_gpu()
# Initialize the Caffe model using the model trained in DIGITS
net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

# Load the input image into a numpy array and display it
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

# Calculate how many 256x256 grid squares are in the image
rows = input_image.shape[0]/256
cols = input_image.shape[1]/256

# Subtract the mean image
for i in range(0,rows):
    for j in range(0,cols):
        input_image[i*256:(i+1)*256,j*256:(j+1)*256] -= mean_image
        
# Initialize an empty array for the detections
detections = np.zeros((rows,cols))
        
# Iterate over each grid square using the model to make a class prediction
start = time.time()
for i in range(0,rows):
    for j in range(0,cols):
        grid_square = input_image[i*256:(i+1)*256,j*256:(j+1)*256]
        # make prediction
        prediction = net.predict([grid_square])
        detections[i,j] = prediction[0].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections)
plt.show()

# Display total time to perform inference
print 'Total inference time (sliding window without overlap): ' + str(end-start) + ' seconds'

# define the amount of overlap between grid cells
OVERLAP = 0.25
grid_rows = int((rows-1)/(1-OVERLAP))+1
grid_cols = int((cols-1)/(1-OVERLAP))+1

print "Image has %d*%d blocks of 256 pixels" % (rows, cols)
print "With overlap=%f grid_size=%d*%d" % (OVERLAP, grid_rows, grid_cols)

# Initialize an empty array for the detections
detections = np.zeros((grid_rows,grid_cols))

# Iterate over each grid square using the model to make a class prediction
start = time.time()
for i in range(0,grid_rows):
    for j in range(0,grid_cols):
        start_col = int(j*256*(1-OVERLAP))
        start_row = int(i*256*(1-OVERLAP))
        grid_square = input_image[start_row:start_row+256, start_col:start_col+256]
        # make prediction
        prediction = net.predict([grid_square])
        detections[i,j] = prediction[0].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections)
plt.show()

# Display total time to perform inference
print ('Total inference time (sliding window with %f%% overlap: ' % (OVERLAP*100)) + str(end-start) + ' seconds'

# now with batched inference (one column at a time)
# we are not using a caffe.Classifier here so we need to do the pre-processing
# manually. The model was trained on random crops (256*256->227*227) so we
# need to do the cropping below. Similarly, we need to convert images
# from Numpy's Height*Width*Channel (HWC) format to Channel*Height*Width (CHW) 
# Lastly, we need to swap channels from RGB to BGR
net = caffe.Net(MODEL_FILE, PRETRAINED, caffe.TEST)
start = time.time()
net.blobs['data'].reshape(*[grid_cols, 3, 227, 227])

# Initialize an empty array for the detections
detections = np.zeros((rows,cols))

for i in range(0,rows):
    for j in range(0,cols):
        grid_square = input_image[i*256:(i+1)*256,j*256:(j+1)*256]
        # add to batch
        grid_square = grid_square[14:241,14:241] # 227*227 center crop        
        image = np.copy(grid_square.transpose(2,0,1)) # transpose from HWC to CHW
        image = image * 255 # rescale
        image = image[(2,1,0), :, :] # swap channels
        net.blobs['data'].data[j] = image
    # make prediction
    output = net.forward()[net.outputs[-1]]
    for j in range(0,cols):
        detections[i,j] = output[j].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections)
plt.show()

# Display total time to perform inference
print 'Total inference time (batched inference): ' + str(end-start) + ' seconds'

完了したら、[ラボに戻ってください。](#question-optional-exercise)

<a id='gu'></a>
# 全畳み込みネットワークの解答

AlexNet 内の **すべて** のテキストを以下と置き換えて、それを全畳み込みネットワークに変換することができます。
コピーしたら、[ここ](#fcn) をクリックしてラボに戻ってください。

```# AlexNet
name: "AlexNet"
layer {
  name: "train-data"
  type: "Data"
  top: "data"
  top: "label"
  transform_param {
    mirror: true
    crop_size: 227
  }
  data_param {
    batch_size: 128
  }
  include { stage: "train" }
}
layer {
  name: "val-data"
  type: "Data"
  top: "data"
  top: "label"
  transform_param {
    crop_size: 227
  }
  data_param {
    batch_size: 32
  }
  include { stage: "val" }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 96
    kernel_size: 11
    stride: 4
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0
    }
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "conv1"
  top: "conv1"
}
layer {
  name: "norm1"
  type: "LRN"
  bottom: "conv1"
  top: "norm1"
  lrn_param {
    local_size: 5
    alpha: 0.0001
    beta: 0.75
  }
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "norm1"
  top: "pool1"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "conv2"
  type: "Convolution"
  bottom: "pool1"
  top: "conv2"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 256
    pad: 2
    kernel_size: 5
    group: 2
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu2"
  type: "ReLU"
  bottom: "conv2"
  top: "conv2"
}
layer {
  name: "norm2"
  type: "LRN"
  bottom: "conv2"
  top: "norm2"
  lrn_param {
    local_size: 5
    alpha: 0.0001
    beta: 0.75
  }
}
layer {
  name: "pool2"
  type: "Pooling"
  bottom: "norm2"
  top: "pool2"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "conv3"
  type: "Convolution"
  bottom: "pool2"
  top: "conv3"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 384
    pad: 1
    kernel_size: 3
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0
    }
  }
}
layer {
  name: "relu3"
  type: "ReLU"
  bottom: "conv3"
  top: "conv3"
}
layer {
  name: "conv4"
  type: "Convolution"
  bottom: "conv3"
  top: "conv4"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 384
    pad: 1
    kernel_size: 3
    group: 2
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu4"
  type: "ReLU"
  bottom: "conv4"
  top: "conv4"
}
layer {
  name: "conv5"
  type: "Convolution"
  bottom: "conv4"
  top: "conv5"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 256
    pad: 1
    kernel_size: 3
    group: 2
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu5"
  type: "ReLU"
  bottom: "conv5"
  top: "conv5"
}
layer {
  name: "pool5"
  type: "Pooling"
  bottom: "conv5"
  top: "pool5"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "conv6"
  type: "Convolution"
  bottom: "pool5"
  top: "conv6"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 4096
    pad: 0
    kernel_size: 6
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu6"
  type: "ReLU"
  bottom: "conv6"
  top: "conv6"
}
layer {
  name: "drop6"
  type: "Dropout"
  bottom: "conv6"
  top: "conv6"
  dropout_param {
    dropout_ratio: 0.5
  }
}
layer {
  name: "conv7"
  type: "Convolution"
  bottom: "conv6"
  top: "conv7"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 4096
    kernel_size: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu7"
  type: "ReLU"
  bottom: "conv7"
  top: "conv7"
}
layer {
  name: "drop7"
  type: "Dropout"
  bottom: "conv7"
  top: "conv7"
  dropout_param {
    dropout_ratio: 0.5
  }
}
layer {
  name: "conv8"
  type: "Convolution"
  bottom: "conv7"
  top: "conv8"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 2
    kernel_size: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "accuracy"
  type: "Accuracy"
  bottom: "conv8"
  bottom: "label"
  top: "accuracy"
  include { stage: "val" }
}
layer {
  name: "loss"
  type: "SoftmaxWithLoss"
  bottom: "conv8"
  bottom: "label"
  top: "loss"
  exclude { stage: "deploy" }
}
layer {
  name: "softmax"
  type: "Softmax"
  bottom: "conv8"
  top: "softmax"
  include { stage: "deploy" }
}
```

コピー後、[ここ](#fcn) をクリックしてラボに戻ってください。